## SIMOC B2 Planning

This file contains the working versions of the data_files to be used in SIMOC-B2, and a basic test of functionality.

Data files include:
* Parameters for agents and currencies (mirroring the current): `agent_desc`, `agent_conn`, `currency_desc`, `agent_events`, and `agent_variation`
* Preset game configurations (new): `mission_1a`, `mission_1b`, `mission_2`

In [13]:
import json

"""
agent_desc
==========
Complete agent list for the first release of SIMOC-B2. 

Some of these we already have agents defined for, 
but whether or not we can reuse/modify the current is TBD.
"""
b2_agent_desc = {
    'inhabitants': {
        'human_agent': {},
        'predators': {},
    },
    'eclss': {
        'b2_co2_removal': {},
        'b2_o2_resupply': {},
    },
    'plants': {        
        'rice': {},
        'wheat': {},
        'sorghum': {},
        'sweet_potato': {},
        'vegetables': {},
        'soybean': {},
        'peanut': {},
        'corn': {},
        'dry_bean': {},
        'orchard': {},
    },
    'structures': {
        'b2_greenhouse': {},
        'b2_crew_quarters': {},
        'b2_biomes': {},
    },
}

# Print the agents we currently have, and those that are missing
current_agents = []
missing_agents = []
with open('data_files/agent_desc.json') as f:
    agent_desc = json.load(f)
    for category, agents in b2_agent_desc.items():
        if category not in agent_desc.keys():
            missing_agents += list(agents.keys())
        for agent, _ in agents.items():
            if agent not in agent_desc[category].keys():
                missing_agents.append(agent)
            else:
                current_agents.append(agent)
print(f'{current_agents=}')
print(f'{missing_agents=}')

current_agents=['human_agent', 'rice', 'wheat', 'sweet_potato', 'soybean', 'peanut', 'dry_bean']
missing_agents=['predators', 'b2_co2_removal', 'b2_o2_resupply', 'sorghum', 'vegetables', 'corn', 'orchard', 'b2_greenhouse', 'b2_crew_quarters', 'b2_biomes']
